## ***מחברת קוד לעיבוד נתוני תוצאות קייטיאי - סקר קרקע***

<bdi>

ברירת המחדל של הטבלה הינה הצגת חריגות מערך ה-VSL.

ניתן להציג חריגות מערך סף נוסף בהתאם למאפייני האתר.

בתא השני נבחר את התרכובות אותם נרצה להציג בטבלת התוצאות. לדוגמה: TPH, MTBE BTEX וכו'.

סימון האופציה **Add_exceptations** יציג בטבלה את כל תרכובות ה-VOC בהם יש חריגות מערכי הסף הנבחרים, VSL ו-TIER 1.

ב-**Add_specifics** נוסיף מספרי CAS של תרכובות נוספות אותן נרצה להציג על אף שלא אותרו בהן חריגות מערכי הסף.


על מנת לעשות זאת, יש לרשום במקום המיועד לכך לפי הפורמט הבא:

'cas number, cas number'

 לדוגמה: '106-93-4, 74-83-9'

 ניתן הוסיף כמה אנליזות שרוצים, 1,2,3 וכו'.

 אם תבוקש אנליזה שלא נמצאת בתוצאות מעבדה תקפוץ הודעה, יש ללחוץ אישור ולבדוק איזה אנליזה לא באמת קיימת בקובץ תוצאות שהעלתם

**יש לשים לב כי שומרים בשם את קובץ התוצאות מהמעבדה לקובץ אקסל רגיל, מהמעבדה זה מגיע בפורמט מוזר, ורק אז להעלות את קובץ האקסל לכאן, אחרת זה לא יעבוד**
</bdi>

In [52]:
#@title נא לבחור פרמטרים מתאימים  { run: "auto" }

Tier1_type = "Tier 1 Residential A >6m" # @param ["Tier 1 Residential A 0-6m","Tier 1 Residential A >6m", "Tier 1 Residential B","Tier 1 industrial A 0-6m","Tier 1 industrial A >6m","Tier 1 industrial B"]

testsׁ_basic = "TPH + Metals" # @param ["TPH","MTBE BTEX","TPHּּּ + MTBE BTEX", "Metals", "TPH + Metals"]

Add_exceptations = True # @param {type:"boolean"}

Add_specifics = None # @param {type:"raw"}

file_name = 'ניסיון' # @param {type:"string"}

Font_size_hebrew = 9 # @param {type:"slider", min:6, max:24, step:1}

Font_name_hebrew = 'David' # @param {type:"string"}

Font_size_english = 8 # @param {type:"slider", min:6, max:24, step:1}

Font_name_english = 'Times New Roman' # @param {type:"string"}

!pip install openpyxl
from openpyxl.styles import Font
import pandas as pd
import re
from IPython.display import Javascript
import numpy as np

if Add_specifics == None:
  Add_specifics = ''


In [2]:
#@title להעלות קובץ ערכי סף של קרקע בלחיצה על הכפתור למטה
from google.colab import files
uploaded = files.upload()

filenames = list(uploaded.keys())

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through each file and read it based on its extension
for filename in filenames:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(filename)
    else:
        print(f"Unsupported file format: {filename}")
        continue
        # Round all numeric cells to 2 decimal points
    #df = df.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)

    dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    threshold_data = pd.concat(dataframes, ignore_index=True)
else:
    print("No valid files uploaded.")


Saving גרסה 7 דצמבר 2024 - soil_threshold.csv to גרסה 7 דצמבר 2024 - soil_threshold (1).csv


In [50]:
#@title להעלות קובץ טבלת תוצאות של קרקע בלחיצה על הכפתור למטה

from google.colab import files
import pandas as pd

# Upload files
uploaded = files.upload()

# List of uploaded filenames
filenames = list(uploaded.keys())

# Initialize empty lists to store dataframes for TPH, IPC, and VOC splits
dataframes_all = []
dataframes_splits = []

# Loop through each file and process based on extension
for filename in filenames:
    if filename.endswith(('.xlsx', '.xls')):
        # Read the first sheet
        try:
            df_all = pd.read_excel(filename, sheet_name='Client SOIL - 1')

            dataframes_all.append(df_all)
        except Exception as e:
            print(f"Error reading first sheet of {filename}: {e}")

        # # Read the second sheet if it exists
        # try:
        #     df_2 = pd.read_excel(filename, sheet_name='Client SOIL - 2', header=None)
        #     # Append the DataFrame to  list
        #     dataframes_all.append(df_2)
        # except Exception as e:
        #     print(f"No second sheet of {filename}: {e}")


# Concatenate all TPH sheet dataframes into one
if dataframes_all:
    results_data = pd.concat(dataframes_all, ignore_index=True)
else:
    print("No valid sheets found.")
    results_data = pd.DataFrame()




# Convert the data to string to avoid errors
results_data = results_data.astype(str)

split_indices = results_data[results_data.astype(str).apply(lambda x: x.str.contains('Client Sample ID', na=False, case=False)).any(axis=1)].index.tolist()

# Split the DataFrame based on detected indices
for i in range(len(split_indices)):
    start_idx = split_indices[i]
    end_idx = split_indices[i + 1] if i + 1 < len(split_indices) else len(results_data)

    # Extract the chunk and reset index
    results_data = results_data.iloc[start_idx:end_idx].reset_index(drop=True)


# # Find all row indices where 'Client Sample ID' appears
# split_indices = results_data[results_data.astype(str).apply(lambda x: x.str.contains('Client Sample ID', na=False, case=False)).any(axis=1)].index.tolist()

# # Initialize list to store split DataFrames
# df_list = []

# # Split the DataFrame based on detected indices
# for i in range(len(split_indices)):
#     start_idx = split_indices[i]
#     end_idx = split_indices[i + 1] if i + 1 < len(split_indices) else len(results_data)

#     # Extract the chunk and reset index
#     df_chunk = results_data.iloc[start_idx:end_idx].reset_index(drop=True)
#     df_list.append(df_chunk)

Saving PR2507576_0_EXCEL_GENERIC.XLS.xlsx to PR2507576_0_EXCEL_GENERIC.XLS.xlsx


In [53]:
#@title יש להריץ את התא ולהוריד את הטבלה המוכנה

# Function to remove '(' and ')' from strings
def remove_parentheses(cell):
    if isinstance(cell, str):  # Only apply to strings
        return cell.replace('(', '').replace(')', '')
    return cell

# Apply to the entire dataframe for results_data
results_data = results_data.applymap(remove_parentheses)


# Search for the row containing 'Client Sample ID'
row_index = results_data[results_data.astype(str).apply(lambda x: x.str.contains('Client Sample ID', na=False, case=False)).any(axis=1)].index

# Initialize an empty list to store the extracted drill names
drills_list = []

if not row_index.empty:
    # Get the first occurrence of the row containing 'Client Sample ID'
    target_row_index = row_index[0]

    # Extract all non-null values from the row, starting from the 'Client Sample ID' column
    row_values = results_data.iloc[target_row_index].dropna().tolist()

    # Store extracted values into drills_list, excluding 'Client Sample ID'
    drills_list.extend(row_values)
    if "Client Sample ID" in drills_list:
        drills_list.remove("Client Sample ID")

# Extract non-null and non-string 'nan' values
drills_list = [val for val in drills_list if str(val).lower() != 'nan']


# Initialize metals_list
metals_list = []

first_column = results_data.iloc[:, 0].fillna("")

# Search for "Extractable Metals / Major Cations" in the first column
match_index = first_column[first_column.str.strip() == "Extractable Metals / Major Cations"].index

# Ensure at least one occurrence is found
if not match_index.empty:
    start_index = match_index[0]  # Take the first occurrence

    # Collect all values below it until a NaN is encountered
    for i in range(start_index + 1, len(results_data)):
        cell_value = results_data.iloc[i, 0]
        if pd.isna(cell_value) or cell_value == "":  # Stop at NaN or empty cell
            break
        metals_list.append(cell_value)


# Define the structure of the DataFrame
columns = ['name', 'date', 'Unnamed: 2', 'depth', ' PID','full name']
data = [
    ['units', None, None, 'm', 'ppm', None],
    ['Cas', None, None, None, None, None],
    ['VSL', None, None, None, None, None],
    ['TIER 1', None, None, None, None, None]
]

# # Normalize 'dup', 'duplicate', or 'Duplicate' to 'DUP'
# normalized_drills_list = []
# for item in drills_list:
#     # Replace 'dup', 'duplicate', or 'Duplicate' with 'DUP' (case-insensitive)
#     normalized_item = re.sub(r'\b(?:dup|duplicate|Duplicate)\b', 'DUP', item, flags=re.IGNORECASE)
#     normalized_drills_list.append(normalized_item)

# # Remove duplicates by converting to a set, then back to a list to maintain unique items
# normalized_drills_list = list(set(normalized_drills_list))

# # Sort the list if you want to keep it in a specific order
# normalized_drills_list.sort()
# drills_list = normalized_drills_list

# Create the DataFrame
df_template = pd.DataFrame(data, columns=columns)

# # Process each item in drills_list
# df_template['full name'] = None
# for item in drills_list:
#     # Find the first instance of a number with a dot and numbers on both sides
#     match = re.search(r'(\d+\.\d+)', item)
#     if match:
#         # Extract the depth (value) and the name (key)
#         depth = match.group(1)  # The matched number with a dot
#         name = item[:match.start()-1].strip()  # Get everything before the depth
#         new_row = {'name': name, 'depth': depth, 'full name' : item}  # Create a new row with name and depth
#         df_template = pd.concat([df_template, pd.DataFrame([new_row])], ignore_index=True)
# # Fill missing columns with None to match the template structure
# df_template = df_template.reindex(columns=columns, fill_value=None)

# # Create the DataFrame
# df_template = pd.DataFrame(data, columns=columns)

# Process each item in drills_list
df_template['full name'] = None
for item in drills_list:
    # Find the first instance of a number with a dot and optional ' DUP' suffix
    match = re.search(r'(\d+\.\d+(?: DUP)?)', item)
    if match:
        # Extract the depth (value) and the name (key)
        depth = match.group(1)  # The matched number with a dot and optional ' DUP'
        name = item[:match.start()-1].strip()  # Get everything before the depth
        new_row = {'name': name, 'depth': depth, 'full name': item}  # Create a new row with name and depth
        df_template = pd.concat([df_template, pd.DataFrame([new_row])], ignore_index=True)

# Fill missing columns with None to match the template structure
df_template = df_template.reindex(columns=columns, fill_value=None)

# Define the starting row for sorting
start_row = 4

# Extract the header rows (rows before start_row) and the rows to be sorted
header_rows = df_template.iloc[:start_row]
rows_to_sort = df_template.iloc[start_row:]

# Add sorting keys for letter part and number part of the name
rows_to_sort['name_letter'] = rows_to_sort['name'].str.extract(r'([A-Za-z]+)', expand=False)
rows_to_sort['name_number'] = rows_to_sort['name'].str.extract(r'(\d+)', expand=False).astype(float)

# Sort the rows based on 'name_letter', 'name_number', and 'depth'
sorted_rows = rows_to_sort.sort_values(
    by=['name_letter', 'name_number', 'depth'],
    key=lambda col: pd.to_numeric(col, errors='coerce') if col.name in ['name_number', 'depth'] else col
)

# Drop the temporary columns used for sorting
sorted_rows = sorted_rows.drop(columns=['name_letter', 'name_number'])

# Concatenate the header rows with the sorted rows
df_sorted = pd.concat([header_rows, sorted_rows], ignore_index=True)

####סידור וניקוי נתונים####

# Remove rows where all cells are NaN or empty
# Replace different types of empty values with actual NaN
results_data = results_data.replace(["", " ", "nan", "NaN", "None"], pd.NA)
results_data_cleaned = results_data.dropna(how='all')

# Transpose the DataFrame
results_data_op = results_data_cleaned.T

# Reindex the DataFrame
results_data_op = results_data_op.reset_index(drop=True)

# Drop rows 1-3 by index
results_data_op = results_data_op.drop(index=[1, 2, 3]).reset_index(drop=True)

# Drop columns 1-4 by index
results_data_op = results_data_op.drop(results_data_op.columns[[1,2,3,4]], axis=1)

# Identify columns where all values from row 1 onward are NaN
cols_to_drop = [col for col in results_data_op.columns if results_data_op.loc[1:, col].isna().all()]

# Drop those columns
results_data_op = results_data_op.drop(columns=cols_to_drop)

# Rename columns numerically
results_data_op.columns = range(results_data_op.shape[1])

# Set the value of row 0, column 0 to 'Analyte'
results_data_op.iat[0, 0] = 'Analyte'

# Set row 0 as the new headers
results_data_op.columns = results_data_op.iloc[0]  # Assign first row as column names
results_data_op = results_data_op[1:].reset_index(drop=True)  # Remove the first row from data

def merge_duplicate_columns(df):
    new_df = pd.DataFrame()  # Create an empty DataFrame to store merged columns

    for col in df.columns.unique():  # Iterate over unique column names
        duplicate_cols = df.loc[:, df.columns == col]  # Get all columns with the same name

        if duplicate_cols.shape[1] > 1:  # If there are duplicates
            merged_col = duplicate_cols.bfill(axis=1).iloc[:, 0]  # Backfill missing values and keep first non-null
            new_df[col] = merged_col  # Store merged column
        else:
            new_df[col] = df[col]  # Keep single columns as they are

    return new_df

# Merge duplicate columns in df_sorted
results_data_op = merge_duplicate_columns(results_data_op)

########הכנסת נתונים לטמפלייט########
if 'TPH' in testsׁ_basic:
  df_sorted['TPH DRO'] = None
  df_sorted['TPH ORO'] = None
  df_sorted['Total TP'] = None

  # Ensure column names are consistent and merge based on "full name" and "Analyte"
  merged_df = df_sorted.merge(
      results_data_op.rename(columns={"Analyte": "full name"}),
      on="full name",
      how="left"
  )

  # Function to handle numeric conversion and '<' values properly
  def clean_numeric(value):
      if isinstance(value, str):
          value = value.strip()
          if value.startswith("<"):  # If the value starts with '<', extract the number and mark it
              try:
                  return f"<{float(value[1:])}"  # Keep '<' format but extract number
              except ValueError:
                  return None  # If conversion fails, return None
          try:
              return float(value)  # Convert to float if possible
          except ValueError:
              return None  # If conversion fails, return None
      return value  # Return original if it's already numeric

  # Convert relevant columns to numeric while keeping '<' format
  merged_df['C10 - C28 Fraction DRO Numeric'] = merged_df['C10 - C28 Fraction DRO'].apply(clean_numeric)
  merged_df['C24 - C40 Fraction ORO Numeric'] = merged_df['C24 - C40 Fraction ORO'].apply(clean_numeric)

  # Function to calculate 'Total TP' based on logic
  def calculate_total_tph(row):
      dro = row['C10 - C28 Fraction DRO Numeric']
      oro = row['C24 - C40 Fraction ORO Numeric']

      if isinstance(dro, str) and dro.startswith("<") and isinstance(oro, str) and oro.startswith("<"):
          # If both are '<' values, sum them as '<total_value'
          try:
              dro_value = float(dro[1:])
              oro_value = float(oro[1:])
              return f"<{dro_value + oro_value}"
          except ValueError:
              return '-'  # If extraction fails, return '-'

      elif isinstance(dro, str) and dro.startswith("<"):
          return oro if isinstance(oro, (int, float)) else dro  # Return numeric value if possible, else keep '<' value

      elif isinstance(oro, str) and oro.startswith("<"):
          return dro if isinstance(dro, (int, float)) else oro  # Return numeric value if possible, else keep '<' value

      elif dro is None and oro is None:
          return '-'  # If both are non-numeric, return '-'

      elif dro is None:
          return oro  # If only DRO is None, return ORO

      elif oro is None:
          return dro  # If only ORO is None, return DRO

      else:
          return dro + oro  # If both are numeric, sum them

  # Apply the function to calculate 'Total TP'
  df_sorted['Total TP'] = merged_df.apply(calculate_total_tph, axis=1)

  # Update the relevant columns in df_sorted
  df_sorted['TPH DRO'] = merged_df['C10 - C28 Fraction DRO']
  df_sorted['TPH ORO'] = merged_df['C24 - C40 Fraction ORO']

  # Renaming the column 'Total TP' to 'Total TPH'
  df_sorted.rename(columns={'Total TP': 'Total TPH'}, inplace=True)

  #Adding CAS number
  df_sorted['TPH DRO'][1] = "C10-C40"
  df_sorted['TPH ORO'][1] = "C10-C40"
  df_sorted['Total TPH'][1] = "C10-C40"


cas_dict = {
    "Benzene": "71-43-2",     "Ethylbenzene": "100-41-4",     "Sum of xylenes": "1330-20-7",     "Toluene": "108-88-3",     "meta- & para-Xylene": "108-38-3",     "ortho-Xylene": "95-47-6",
    "Benzyl Alcohol": "100-51-6",     "6-Caprolactam": "105-60-2",     "Acetophenone": "98-86-2",     "Isophorone": "78-59-1",     "4-Chloroaniline": "106-47-8",     "Aniline": "62-53-3",
    "Benzidine": "92-87-5",     "Diphenylamine": "122-39-4",     "1,1`-Biphenyl": "92-52-4",     "1-Chloronaphthalene": "nan",     "2-Chloronaphthalene": "91-58-7",     "1-Methylnaphthalene": "90-12-0",
    "2-Methylnaphthalene": "91-57-6",     "4-Bromophenyl phenyl ether": "nan",     "4-Chlorophenyl phenyl ether": "nan",     "Carbazole": "nan",     "Dibenzofuran": "132-64-9",     "1.1-Dichloroethane": "75-34-3",
    "1.1-Dichloroethene": "nan",     "1.1-Dichloropropene": "nan",     "1.1.1-Trichloroethane": "71-55-6",     "1.1.1.2-Tetrachloroethane": "630-20-6",     "1.1.2-Trichloroethane": "79-00-5",     "1.1.2.2-Tetrachloroethane": "79-34-5",
    "1.2-Dibromo-3-chloropropane": "96-12-8",     "1.2-Dibromoethane EDB": "106-93-4",     "1.2-Dichlorobenzene": "95-50-1",     "1.2-Dichloroethane": "107-06-2",     "1.2-Dichloropropane": "78-87-5",     "1.2.3-Trichlorobenzene": "87-61-6",
    "1.2.3-Trichloropropane": "96-18-4",     "1.2.4-Trichlorobenzene": "87-61-6",     "1.3-Dichlorobenzene": "nan",     "1.3-Dichloropropane": "142-28-9",     "1.4-Dichlorobenzene": "106-46-7",     "2-Chlorotoluene": "95-49-8",
    "4-Chlorotoluene": "106-43-4",     "2.2-Dichloropropane": "nan",     "Bromobenzene": "108-86-1",     "Bromochloromethane": "74-97-5",     "Bromodichloromethane": "75-27-4",     "Bromoform": "75-25-2",
    "Bromomethane": "74-83-9",     "Chlorobenzene": "108-90-7",     "Chloroethane": "75-00-3",     "Chloroform": "67-66-3",     "Chloromethane": "74-87-3",     "Dibromochloromethane": "124-48-1",
    "Dibromomethane": "74-95-3",     "Dichlorodifluoromethane": "75-71-8",     "Dichloromethane": "nan",     "Hexachlorobutadiene": "87-68-3",     "Tetrachloroethene": "nan",     "Tetrachloromethane": "nan",
    "Trichloroethene": "nan",     "Vinyl chloride": "75-01-4",     "cis-1.3-Dichloropropene": "542-75-6",     "trans-1.3-Dichloropropene": "542-75-6",     "cis-1.2-Dichloroethene": "nan",     "trans-1.2-Dichloroethene": "nan",
    "1.2.4-Trimethylbenzene": "95-63-6",     "1.3.5-Trimethylbenzene": "108-67-8",     "1.4-Dioxane": "123-91-1",     "2-Butanone MEK": "78-93-3",     "Acetone": "67-64-1",     "Isopropylbenzene": "98-82-8",
    "Methyl isobutyl ketone": "108-10-1",     "Methyl tert-Butyl Ether MTBE": "1634-04-4",     "Styrene": "100-42-5",     "n-Butylbenzene": "104-51-8",     "n-Propylbenzene": "nan",     "p-Isopropyltoluene": "nan",
    "sec-Butylbenzene": "135-98-8",     "tert-Butyl alcohol": "nan",     "tert-Butylbenzene": "98-06-6",     "1.2.3.4-Tetrachlorobenzene": "nan",     "1.2.3.5- & 1.2.4.5-Tetrachlorobenzene": "95-94-3",     "2.4-DDD": "nan",
    "4.4`-DDD": "72-54-8",     "2.4-DDE": "nan",     "2.4-DDT": "50-29-3",     "4.4`-DDE": "72-55-9",     "4.4`-DDT": "50-29-3",     "Alachlor": "15972-60-8",
    "Aldrin": "309-00-2",     "Chlordane-cis": "12789-03-6",     "Chlordane-trans": "12789-03-6",     "Dieldrin": "60-57-1",     "Endosulfan sulfate": "115-29-7",     "Endrin": "72-20-8",
    "Heptachlor": "76-44-8",     "Heptachloroepoxide-cis": "1024-57-3",     "Heptachloroepoxide-trans": "1024-57-3",     "Hexachlorobenzene HCB": "118-74-1",     "Hexachlorocyclohexane Alpha": "319-84-6",     "Hexachlorocyclohexane Beta": "319-85-7",
    "Hexachlorocyclohexane Delta": "nan",     "Hexachlorocyclohexane Epsilon": "nan",     "Hexachlorocyclohexane Gamma": "58-89-9",     "Hexachloroethane": "67-72-1",     "Isodrin": "nan",     "Methoxychlor": "72-43-5",
    "Mirex": "2385-85-5",     "Nonachlor-cis": "nan",     "Nonachlor-trans": "nan",     "Oxychlordane": "nan",     "Pentachlorobenzene": "608-93-5",     "Sum of 3 tetrachlorobenzenes": "nan",
    "Sum of 3 tetrachlorobenzenes M1": "nan",     "Sum of 4 hexachlorcyclohexanes": "nan",     "Sum of 4 hexachlorocyclohexanes M1": "nan",     "Sum of 4 isomers DDT": "nan",     "Sum of 4 isomers DDT M1": "nan",     "Sum of 6 isomers DDT": "nan",
    "Sum of 6 isomers DDT M1": "nan",     "Telodrin": "nan",     "Trifluralin": "1582-09-8",     "alpha-Endosulfan": "115-29-7",     "beta-Endosulfan": "115-29-7",     "PBB 153": "nan",
    "Dry matter @ 105ֲ°C": "nan",     "Naphthalene": "91-20-3",     "C10 - C28 Fraction DRO": "Diesel Range Organics",     "C24 - C40 Fraction ORO": "Oil Range Organics",     "Bis2-chloroethoxymethane": "111-91-1",     "Bis2-chloroethylether": "111-44-4",
    "Bis2-chloroisopropylether all isomers": "nan",     "2-Chlorophenol": "95-57-8",     "2.4.5-Trichlorophenol": "95-95-4",     "2.4.6-Trichlorophenol": "88-06-2",     "2.4@2.5-Dichlorophenol": "120-83-2",     "2.6-Dichlorophenol": "nan",
    "Pentachlorophenol": "87-86-5",     "2,4-Dimethylphenol": "95-65-8",     "2-Methylphenol": "nan",     "3- & 4-Methylphenol": "nan",     "4-Chloro-3-methylphenol": "nan",     "Phenol": "108-95-2",
    "Aluminium": "7429-90-5",     "Antimony": "7440-36-0",     "Arsenic": "7440-38-2",     "Barium": "7440-39-3",     "Beryllium": "7440-41-7",     "Bismuth": "nan",
    "Boron": "nan",     "Cadmium": "7440-43-9",     "Calcium": "nan",     "Chromium": "7440-47-3",     "Cobalt": "7440-48-4",     "Copper": "7440-50-8",
    "Iron": "7439-89-6", "Lead" : "7439-92-1" ,   "Lithium": "7439-93-2",     "Magnesium": "nan",     "Manganese": "7439-96-5",     "Mercury": "7439-97-6",     "Molybdenum": "7439-98-7",
    "Nickel": "7440-02-0",     "Phosphorus": "7723-14-0",     "Potassium": "nan",     "Selenium": "7782-49-2",     "Silicon": "nan",     "Silver": "7440-22-4",
    "Sodium": "nan",     "Strontium": "7440-24-6",     "Sulphur": "nan",     "Tellurium": "nan",     "Thallium": "7440-28-0",     "Tin": "7440-31-5",
    "Titanium": "7550-45-0",     "Vanadium": "7440-62-2",     "Zinc": "7440-66-6",     "Zirconium": "7440-67-7",     "Hexachlorocyclopentadiene": "77-47-4",     "2,4-Dinitrophenol": "51-28-5",
    "2,4-Dinitrotoluene": "121-14-2",     "2-Nitroaniline": "88-74-4",     "2-Nitrophenol": "nan",     "2.6-Dinitrotoluene": "606-20-2",     "3-Nitroaniline": "nan",     "4,6-Dinitro-2-methylphenol": "nan",
    "4-Nitroaniline": "100-01-6",     "4-Nitrophenol": "nan",     "Nitrobenzene": "98-95-3",     "N-Nitrosodi-n-propylamine": "621-64-7",     "Total SVOC IL M4": "nan",     "Dinoseb": "88-85-7",
    "Bis2-ethylhexylphthalate": "117-81-7",     "Butyl benzyl phthalate": "85-68-7",     "Di-n-butyl phthalate": "84-74-2",     "Di-n-octyl phthalate": "117-84-0",     "Diethyl phthalate": "84-66-2",     "Dimethyl phthalate": "nan",
    "pH H2O": "nan",     "Acenaphthene": "83-32-9",     "Acenaphthylene": "nan",     "Anthracene": "120-12-7",     "Benzaanthracene": "56-55-3",     "Benzoapyrene": "50-32-8",
    "Benzobfluoranthene": "205-99-2",     "Benzog.h.iperylene": "191-24-2",     "Benzokfluoranthene": "207-08-9",     "Chrysene": "218-01-9",     "Dibenza.hanthracene": "53-70-3",     "Fluoranthene": "206-44-0",
    "Fluorene": "86-73-7",     "Indeno1.2.3.cdpyrene": "193-39-5",     "Phenanthrene": "nan",     "Pyrene": "129-00-0",
}

# Function to apply renaming rules robustly
def rename_chemical(name):
    if pd.isna(name):  # Handle NaN values safely
        return name

    # Rule 1: Move numerical prefixes before the chemical name and replace commas between numbers with dots
    pattern1 = r"(\w+), ([\d,.-]+)-"  # Matches "Name, Number(s)-" pattern
    match = re.search(pattern1, name)
    if match:
        base_name, prefix = match.groups()
        formatted_prefix = prefix.replace(",", ".")  # Convert ',' between numbers to '.'
        name = f"{formatted_prefix}-{base_name}"  # Move prefix before the name

    # Rule 2: Handling "p-", "o-", "m-" prefixes (e.g., "Aminophenol, p-" → "4-Aminophenol")
    positional_mapping = {
        "p-": "4-",
        "o-": "2-",
        "m-": "3-"
    }

    pattern2 = r"(\w+), (p-|o-|m-)$"
    match = re.search(pattern2, name)
    if match:
        base_name, prefix = match.groups()
        name = f"{positional_mapping[prefix]}{base_name}"  # Replace with correct numeric prefix

    return name



# Apply renaming rules to the "Chemical" column
threshold_data["Chemical"] = threshold_data["Chemical"].apply(rename_chemical)

import numpy as np
from difflib import get_close_matches

# Create a new row with CAS numbers
cas_row = ["CAS No."]  # First cell as a label

# Extract "Chemical" and "CAS No." from threshold_data for matching
chemicals_threshold = threshold_data[['Chemical', 'CAS No.']].dropna()

# Iterate through each column header in results_data_op (excluding the first column)
for chem in results_data_op.columns[1:]:
    cas_value = cas_dict.get(chem, np.nan)  # Lookup CAS number in dictionary, if not found, set NaN

    # If CAS is NaN, try finding the best match from threshold_data['Chemical']
    if pd.isna(cas_value):
        best_match = get_close_matches(chem, chemicals_threshold['Chemical'], n=1, cutoff=0.8)
        if best_match:
            matched_row = chemicals_threshold[chemicals_threshold['Chemical'] == best_match[0]]
            cas_value = matched_row['CAS No.'].values[0] if not matched_row.empty else np.nan

    cas_row.append(cas_value)

# Insert the CAS No. row at the top of results_data_op
cas_no_df = pd.DataFrame([cas_row], columns=results_data_op.columns)
results_data_op = pd.concat([cas_no_df, results_data_op], ignore_index=True)

# Initialize metals dictionary
metals_dict = {}

# Iterate over each metal in metals_list
for metal in metals_list:
    if metal in results_data_op.columns:  # Check if metal exists in headers
        cas_value = results_data_op.loc[0, metal]  # Get CAS number from row 0
        metals_dict[metal] = cas_value  # Add to dictionary

#Remove the headers names
results_data_op.columns = range(len(results_data_op.columns))

if 'MTBE' in testsׁ_basic:
  df_sorted['MTBE'] = None
  df_sorted['Benzene'] = None
  df_sorted['Toluene'] = None
  df_sorted['Ethylbenzene'] = None
  df_sorted['Xylene'] = None

  #Adding CAS number
  df_sorted['MTBE'][1] = '1634-04-4'
  df_sorted['Benzene'][1] = '71-43-2'
  df_sorted['Toluene'][1] = '108-88-3'
  df_sorted['Ethylbenzene'][1] = '100-41-4'
  df_sorted['Xylene'][1] = '95-47-6'

  # Ensure matching names are correctly formatted
  df_sorted['full name'] = df_sorted['full name'].astype(str).str.strip()
  results_data_op[0] = results_data_op[0].astype(str).str.strip()

  # Extract CAS mappings for the required columns
  cas_map = df_sorted.loc[1, ["MTBE", "Benzene", "Toluene", "Ethylbenzene", "Xylene"]].to_dict()

  # Iterate over each row in df_sorted and fill in the values
  for index, row in df_sorted.iterrows():
      if index <= 1:  # Skip non-data rows
          continue
      name = row['full name']
      if pd.isna(name):
          continue

      # Find the matching name in results_data_op (reversed logic)
      matching_row = results_data_op[results_data_op[0].apply(lambda x: x == name if pd.notna(x) else False)]
      if matching_row.empty:
          continue

      for column, cas in cas_map.items():
          if pd.isna(cas):
              continue

          # Find the column in results_data_VOC_op that matches the CAS
          try:
              cas_column_index = results_data_op.iloc[0, :].tolist().index(cas)  # Exact CAS match in the first row
              value_to_fill = matching_row.iloc[0, cas_column_index]  # Extract the correct value
              df_sorted.at[index, column] = value_to_fill
          except ValueError:
              # CAS not found in the first row
              continue

# Adding metals to df_sorted if 'Metals' is in tests_basic
if 'Metals' in testsׁ_basic:
    # Initialize columns for each metal in df_sorted
    for metal, cas in metals_dict.items():
        df_sorted[metal] = None

    # Add CAS numbers in the second row (index 1) for each metal
    for metal, cas in metals_dict.items():
        df_sorted.at[1, metal] = cas

    # Ensure matching names are correctly formatted
    df_sorted['full name'] = df_sorted['full name'].astype(str).str.strip()

    # Iterate over each row in df_sorted and fill in the values
    for index, row in df_sorted.iterrows():
        if index <= 1:  # Skip non-data rows
            continue
        name = row['full name']
        if pd.isna(name):
            continue

        # Find the matching name in results_data_ICP_op
        matching_row = results_data_op[results_data_op[0].apply(lambda x: x == name if pd.notna(x) else False)]
        if matching_row.empty:
            continue

        # Iterate over metals and fill their values from results_data_ICP_op
        for metal, cas in metals_dict.items():
            try:
                # Find the column in results_data_ICP_op that matches the CAS
                cas_column_index = results_data_op.iloc[0, :].tolist().index(cas)  # Exact CAS match in the first row
                value_to_fill = matching_row.iloc[0, cas_column_index]  # Extract the correct value
                df_sorted.at[index, metal] = value_to_fill
            except ValueError:
                # CAS not found in the first row
                continue

if Add_exceptations == True:
  # Ensure CAS numbers in both dataframes are properly formatted
  results_data_op.iloc[0, :] = results_data_op.iloc[0, :].astype(str).str.strip()
  threshold_data['CAS No.'] = threshold_data['CAS No.'].astype(str).str.strip()

  # Initialize a new row with NaN values
  new_row = [None] * results_data_op.shape[1]

  # Iterate through each CAS number in the first row of results_data_op
  for col_index, cas_number in enumerate(results_data_op.iloc[0, :]):
      if pd.isna(cas_number) or cas_number == 'nan':
          continue

      # Find the matching row in threshold_data
      matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]
      if not matching_row.empty:
          # Get the value from the "VSL" column
          vsl_value = matching_row.iloc[0]['VSL']
          new_row[col_index] = vsl_value

  # Append the new row to results_data_op
  results_data_op.loc[len(results_data_op)] = new_row

  ##adding the values
  # Iterate through each column in results_data_op
  for col_index in range(results_data_op.shape[1]):
      cas_number = results_data_op.iloc[0, col_index]  # Get the CAS number from row 0
      if pd.isna(cas_number) or cas_number == 'nan':
          continue

      # Check if the CAS number is already in df_sorted
      if cas_number in df_sorted.iloc[1, :].values:
          continue  # Skip if the CAS number is already present

      # Get the column values excluding the first row (CAS number)
      column_values = results_data_op.iloc[1:, col_index]

      # Skip columns where all values are NaN
      if column_values.isna().all():
          continue

      # Filter column values to keep only numeric ones
      numeric_values = pd.to_numeric(column_values, errors='coerce')

      # Get the VSL value (assuming it's in the last row of the column)
      vsl_value = results_data_op.iloc[-1, col_index]
      try:
          vsl_value = float(vsl_value)  # Ensure VSL value is a float
      except ValueError:
          continue  # Skip columns where VSL is not a valid number

      # Check if any numeric value in the column exceeds the VSL value
      if (numeric_values > vsl_value).any():
          # Find the row in results_data_VOC where the CAS number matches
          matching_row = threshold_data[
                threshold_data['CAS No.'] == cas_number  # Match CAS number in column 1
            ]

          # If a match is found, get the header from column 0 in the matching row
          if not matching_row.empty:
              header_name = matching_row.iloc[0, -1]
          else:
              header_name = f"Column_{cas_number}"  # Fallback to a generic name

          # Add a new column to df_sorted with the header_name
          df_sorted[header_name] = None
          df_sorted.at[1, header_name] = cas_number

          # Add matching values to df_sorted for the rows where 'full name' matches
          for index, row in df_sorted.iterrows():
              if index <= 1:  # Skip the header and CAS rows
                  continue
              name = row['full name']
              if pd.isna(name):
                  continue

              # Find the matching row in results_data_VOC_op
              matching_row = results_data_op[
                  results_data_op[0].apply(lambda x: x == name if pd.notna(x) else False)
              ]
              if matching_row.empty:
                  continue

              # Extract the value from results_data_op for the current column
              value_to_fill = matching_row.iloc[0, col_index]
              df_sorted.at[index, header_name] = value_to_fill




if Add_specifics:
    Add_specifics_list = [item.strip() for item in Add_specifics.split(',')]
    for item in Add_specifics_list:
        if item not in results_data_op.iloc[0].values:
            display(Javascript('alert("⚠️ one or more of the specific CAS numbers you insert is not in lab results ⚠️")'))
            print(f'{item} is not in lab results')
        else:
            # Find the column index of the item in results_data_op
            try:
                col_index = results_data_op.iloc[0, :].tolist().index(item)
            except ValueError:
                print(f'{item} not found in results_data_op header.')
                continue

            # Check if the CAS number (item) is already in df_sorted
            if item in df_sorted.iloc[1, :].values:
                continue  # Skip if the CAS number is already present

            # Find the row in results_data_VOC where the CAS number matches
            matching_row = threshold_data[
                threshold_data['CAS No.'] == item  # Match CAS number in column 1
            ]

            # If a match is found, get the header from column 0 in the matching row
            if not matching_row.empty:
                header_name = matching_row.iloc[0, -1]
            else:
                header_name = f"Column_{item}"  # Fallback to a generic name

            # Add a new column to df_sorted with the header_name
            df_sorted[header_name] = None
            df_sorted.at[1, header_name] = item

            # Add matching values to df_sorted for the rows where 'full name' matches
            for index, row in df_sorted.iterrows():
                if index <= 1:  # Skip the header and CAS rows
                    continue
                name = row['full name']
                if pd.isna(name):
                    continue

                # Find the matching row in results_data_op
                matching_row = results_data_op[
                    results_data_op[0].apply(lambda x: x == name if pd.notna(x) else False)
                ]
                if matching_row.empty:
                    continue

                # Extract the value from results_data_op for the current column
                value_to_fill = matching_row.iloc[0, col_index]
                df_sorted.at[index, header_name] = value_to_fill


df_sorted.iloc[0, 6:] = 'mg/kg'

# Convert all <NA> values to NaN
df_sorted = df_sorted.astype(object).where(pd.notna(df_sorted), np.nan)

# Iterate through each column in df_sorted starting from column 6 onward
for col_index in range(6, len(df_sorted.columns)):
    cas_number = df_sorted.iat[1, col_index]  # Get the CAS number from row 1

    if pd.isna(cas_number) or cas_number == '':  # Skip if CAS number is empty or NaN
        continue

    # Find the matching row in threshold_data based on CAS No.
    matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]

    if not matching_row.empty:
        # Extract the VSL value from the matching row
        vsl_value = matching_row.iloc[0]['VSL']
        # Populate the cell in row 2 of df_sorted for the current column
        df_sorted.iat[2, col_index] = vsl_value

        # Extract the value from the column with the name Tier1_type
        if Tier1_type in matching_row.columns:
            tier1_value = matching_row.iloc[0][Tier1_type]
            # Populate the cell in row 3 of df_sorted for the current column
            df_sorted.iat[3, col_index] = tier1_value

df_sorted = df_sorted.drop(columns='full name')

########סידור לטבלת אקסל להורדה########
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

# Create an Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active

# Set the sheet to right-to-left
ws.sheet_view.rightToLeft = True

thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

alignment_center_wrap = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Define colors for conditional formatting
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
custom_fill = PatternFill(start_color="F7C7AC", end_color="F7C7AC", fill_type="solid")

# Step 1: Add the merged cells for the headers
ws.merge_cells(start_row=1, start_column=1, end_row=5, end_column=1)
cell = ws.cell(row=1, column=1)
cell.value = "שם קידוח"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

for i in range(2, 6):
  cell = ws.cell(row=i, column=1)
  cell.border = Border(
      left=Side(style='thin'),
      right=Side(style='thin'))

for i in range(2, 6):  # Iterate over columns (2 to 5 inclusive)
    for j in range(1, len(df_sorted)+2):  # Iterate over rows (1 to len(df_sorted) - 1)
        cell = ws.cell(row=j, column=i)
        cell.border = Border(bottom=Side(style='thin'))

ws.merge_cells(start_row=1, start_column=2, end_row=5, end_column=2)
cell = ws.cell(row=1, column=2)
cell.value = "תאריך"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=3)
cell.value = ""
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=3)
cell.value = "יחידות"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=4)
cell.value = "עומק"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=4)
cell.value = "m"
cell.font = Font(name=Font_name_english, size=Font_size_english)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=5)
cell.value = "PID"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=5)
cell.value = "ppm"
cell.font = Font(name=Font_name_english, size=Font_size_english)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=3, start_column=3, end_row=3, end_column=5)
cell = ws.cell(row=3, column=3)
cell.value = "CAS"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=4, start_column=3, end_row=4, end_column=5)
cell = ws.cell(row=4, column=3)
cell.value = "VSL"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=5, start_column=3, end_row=5, end_column=5)
cell = ws.cell(row=5, column=3)
cell.value = "TIER 1"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

# Helper function to detect if a string contains English letters
def contains_english(value):
    return any(char.isascii() and char.isalpha() for char in str(value))

# Step 2: Add the 'name' column to the first column in the Excel file starting from row 4 of df_sorted
for row_idx, value in enumerate(df_sorted["name"][4:], start=6):
    cell = ws.cell(row=row_idx, column=1)
    cell.value = value
    cell.font = Font(
        name=Font_name_english if contains_english(value) else Font_name_hebrew,
        size=Font_size_english if contains_english(value) else Font_size_hebrew,
    )
    cell.alignment = alignment_center_wrap
    cell.border = thin_border

# Merge cells in column 1 with the same value
current_start = 6
previous_value = None
for row_idx, value in enumerate(df_sorted["name"][4:], start=6):
    if value == previous_value:
        continue
    if previous_value is not None:
        ws.merge_cells(start_row=current_start, start_column=1, end_row=row_idx - 1, end_column=1)
    current_start = row_idx
    previous_value = value
# Merge the last group
if previous_value is not None:
    ws.merge_cells(start_row=current_start, start_column=1, end_row=len(df_sorted["name"][4:]) + 5, end_column=1)

# Step 3: Add the 'depth' column to column 4 in the Excel file starting from row 6
for row_idx, value in enumerate(df_sorted["depth"][4:], start=6):
    cell = ws.cell(row=row_idx, column=4)
    cell.value = value
    cell.font = Font(
        name=Font_name_english if contains_english(value) else Font_name_hebrew,
        size=Font_size_english if contains_english(value) else Font_size_hebrew,
    )
    cell.alignment = alignment_center_wrap
    cell.border = thin_border

# Step 4: Add data from column 6 onward starting from row 1 in Excel
for col_idx, col_name in enumerate(df_sorted.columns[5:], start=6):  # Iterate through columns 6 onward
    # Add column header
    header_cell = ws.cell(row=1, column=col_idx)
    header_cell.value = col_name
    header_cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
    header_cell.alignment = alignment_center_wrap
    header_cell.border = thin_border

    # Add column data and round float values
    for row_idx, value in enumerate(df_sorted[col_name], start=2):  # Start from row 2 in Excel
        cell = ws.cell(row=row_idx, column=col_idx)
        try:
            value_numeric = round(float(value), 3)  # Round to 3 decimal places
            cell.value = value_numeric
        except (ValueError, TypeError):
            cell.value = value  # Non-numeric values remain unchanged

        cell.font = Font(
            name=Font_name_english if contains_english(value) else Font_name_hebrew,
            size=Font_size_english if contains_english(value) else Font_size_hebrew,
        )
        cell.alignment = alignment_center_wrap
        cell.border = thin_border

        # Conditional formatting for rows 6 and onward
        if row_idx >= 6 and isinstance(cell.value, (float, int)):
            try:
                row_4_value = float(ws.cell(row=4, column=col_idx).value)
                row_5_value = float(ws.cell(row=5, column=col_idx).value)

                if cell.value > row_4_value:
                    cell.fill = yellow_fill
                    cell.font = Font(bold=True, name=cell.font.name, size=cell.font.size)
                if cell.value > row_5_value:
                    cell.fill = custom_fill
                    cell.font = Font(bold=True, name=cell.font.name, size=cell.font.size)
            except (ValueError, TypeError):
                pass  # Skip cells that cannot be compared numerically

# Merge columns 2 and 3 for each row from row 6 onward
for row_idx in range(6, len(df_sorted["name"]) + 6):
    ws.merge_cells(start_row=row_idx, start_column=2, end_row=row_idx, end_column=3)

# Define the medium border
medium_side = Side(style="medium")

# Define the range for the table
start_row = 1  # Start from the first row
end_row = len(df_sorted)+1  # Dynamically determine the last row based on the data
start_col = 1  # Start from the first column
end_col = ws.max_column  # Dynamically determine the last column based on the data

# Apply medium border to the outer edges of the table
for row in range(start_row, end_row + 1):
    for col in range(start_col, end_col + 1):
        cell = ws.cell(row=row, column=col)

        # Apply borders to the outer edges only
        if row == start_row:  # Top edge
            cell.border = Border(top=medium_side, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
        if row == end_row:  # Bottom edge
            cell.border = Border(bottom=medium_side, left=cell.border.left, right=cell.border.right, top=cell.border.top)
        if col == start_col:  # Left edge
            cell.border = Border(left=medium_side, top=cell.border.top, bottom=cell.border.bottom, right=cell.border.right)
        if col == end_col:  # Right edge
            cell.border = Border(right=medium_side, top=cell.border.top, bottom=cell.border.bottom, left=cell.border.left)

# Define the range for the first 5 rows
start_row = 1
end_row = 5
start_col = 1
end_col = ws.max_column  # Dynamically determine the last column based on the data

# Apply thick border to the outer edges
for row in range(start_row, end_row + 1):
    for col in range(start_col, end_col + 1):
        cell = ws.cell(row=row, column=col)

        # Apply borders to the outer edges only
        if row == start_row:  # Top edge
            cell.border = Border(top=medium_side, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
        if row == end_row:  # Bottom edge
            cell.border = Border(bottom=medium_side, left=cell.border.left, right=cell.border.right, top=cell.border.top)
        if col == start_col:  # Left edge
            cell.border = Border(left=medium_side, top=cell.border.top, bottom=cell.border.bottom, right=cell.border.right)
        if col == end_col:  # Right edge
            cell.border = Border(right=medium_side, top=cell.border.top, bottom=cell.border.bottom, left=cell.border.left)

# Determine the last row and column of the table
last_row = ws.max_row
start_col = 1  # Assuming you want the legend to start from the first column

# Add legend for "חורג מערך VSL"
legend_vsl_cell = ws.cell(row=len(df_sorted)+2 , column=start_col)
legend_vsl_cell.value = "חורג מערך VSL"
legend_vsl_cell.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_vsl_cell.alignment = alignment_center_wrap
legend_vsl_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
legend_vsl_cell.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for "חורג מערך TIER 1"
legend_tier1_cell = ws.cell(row=len(df_sorted)+3 , column=start_col)
legend_tier1_cell.value = "חורג מערך TIER 1"
legend_tier1_cell.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_tier1_cell.alignment = alignment_center_wrap
legend_tier1_cell.fill = PatternFill(start_color="F7C7AC", end_color="F7C7AC", fill_type="solid")
legend_tier1_cell.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Save the final Excel file with the provided name
file_path = f'{file_name}.xlsx'
wb.save(file_path)

# Download the Excel file
files.download(file_path)


<ipython-input-53-34d7492877de>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_data = results_data.applymap(remove_parentheses)
<ipython-input-53-34d7492877de>:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_to_sort['name_letter'] = rows_to_sort['name'].str.extract(r'([A-Za-z]+)', expand=False)
<ipython-input-53-34d7492877de>:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_to_sort['name_number'] = rows_to_sort['name'].str.extract(r'(\d+)', exp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>